# Epic 1.0 Services - Interactive Testing Notebook

This notebook allows you to interactively test the three core services:
- **CalculationEngineService**
- **CompletenessValidationService**  
- **ComplianceValidationService**

Run each cell to see the services in action!

## Setup - Import Services

In [ ]:
from app.services.calculation_engine_service import calculation_engine_service
from app.services.completeness_validation_service import completeness_validation_service
from app.services.compliance_validation_service import compliance_validation_service
from app.db.enums import ValidationStatus
import json

print("✅ Services imported successfully!")

## Test 1: Calculation Engine - Simple Threshold

In [ ]:
# Define a calculation schema
calculation_schema = {
    "condition_groups": [
        {
            "operator": "AND",
            "rules": [
                {
                    "rule_type": "PERCENTAGE_THRESHOLD",
                    "field_id": "completion_rate",
                    "operator": ">=",
                    "threshold": 75.0,
                    "description": "Completion rate must be at least 75%"
                }
            ]
        }
    ],
    "output_status_on_pass": "Pass",
    "output_status_on_fail": "Fail"
}

# Try different completion rates
test_cases = [
    {"completion_rate": 85.0},   # Should PASS
    {"completion_rate": 75.0},   # Should PASS (exactly at threshold)
    {"completion_rate": 60.0},   # Should FAIL
]

for response_data in test_cases:
    result = calculation_engine_service.execute_calculation(
        calculation_schema, response_data
    )
    rate = response_data['completion_rate']
    icon = "✅" if result == ValidationStatus.PASS else "❌"
    print(f"{icon} Completion Rate: {rate}% → {result.value}")

## Test 2: Calculation Engine - Multiple Conditions (AND)

In [ ]:
# Schema with multiple conditions that must ALL pass
complex_schema = {
    "condition_groups": [
        {
            "operator": "AND",
            "rules": [
                {
                    "rule_type": "PERCENTAGE_THRESHOLD",
                    "field_id": "completion_rate",
                    "operator": ">=",
                    "threshold": 75.0
                },
                {
                    "rule_type": "COUNT_THRESHOLD",
                    "field_id": "required_documents",
                    "operator": ">=",
                    "threshold": 3
                },
                {
                    "rule_type": "MATCH_VALUE",
                    "field_id": "approval_status",
                    "operator": "==",
                    "expected_value": "approved"
                }
            ]
        }
    ],
    "output_status_on_pass": "Pass",
    "output_status_on_fail": "Fail"
}

# Test scenarios
scenarios = [
    {
        "name": "All conditions met",
        "data": {
            "completion_rate": 85.0,
            "required_documents": ["doc1", "doc2", "doc3"],
            "approval_status": "approved"
        }
    },
    {
        "name": "Missing documents",
        "data": {
            "completion_rate": 85.0,
            "required_documents": ["doc1", "doc2"],  # Only 2
            "approval_status": "approved"
        }
    },
    {
        "name": "Not approved",
        "data": {
            "completion_rate": 85.0,
            "required_documents": ["doc1", "doc2", "doc3"],
            "approval_status": "pending"  # Not approved
        }
    }
]

for scenario in scenarios:
    result = calculation_engine_service.execute_calculation(
        complex_schema, scenario["data"]
    )
    icon = "✅" if result == ValidationStatus.PASS else "❌"
    print(f"{icon} {scenario['name']}: {result.value}")

## Test 3: Remark Generation

In [ ]:
# Define remark templates
remark_schema = {
    "Pass": "✅ Excellent! All requirements met. Your submission is compliant.",
    "Fail": "❌ Requirements not met. Please review the following areas and resubmit.",
    "Conditional": "⚠️ Partially compliant. Additional documentation or clarification needed."
}

# Generate remarks for each status
for status in [ValidationStatus.PASS, ValidationStatus.FAIL, ValidationStatus.CONDITIONAL]:
    remark = calculation_engine_service.get_remark_for_status(remark_schema, status)
    print(f"\n{status.value}:")
    print(f"  {remark}")

## Test 4: Completeness Validation - Basic Form

In [ ]:
# Define a form schema
form_schema = {
    "fields": [
        {
            "field_id": "project_name",
            "field_type": "text_input",
            "label": "Project Name",
            "required": True
        },
        {
            "field_id": "project_description",
            "field_type": "text_area",
            "label": "Project Description",
            "required": True
        },
        {
            "field_id": "budget",
            "field_type": "number_input",
            "label": "Budget (Optional)",
            "required": False
        }
    ]
}

# Test with complete data
complete_data = {
    "project_name": "Community Health Center",
    "project_description": "Building a new health center",
    "budget": 500000
}

result = completeness_validation_service.validate_completeness(form_schema, complete_data)

print("Complete Form Test:")
print(f"  ✅ Is Complete: {result['is_complete']}")
print(f"  📊 Progress: {result['filled_field_count']}/{result['required_field_count']} required fields")

# Test with incomplete data
incomplete_data = {
    "project_name": "Community Health Center"
    # Missing project_description
}

result = completeness_validation_service.validate_completeness(form_schema, incomplete_data)

print("\nIncomplete Form Test:")
print(f"  ❌ Is Complete: {result['is_complete']}")
print(f"  📊 Progress: {result['filled_field_count']}/{result['required_field_count']} required fields")
print(f"  Missing Fields:")
for field in result['missing_fields']:
    print(f"    - {field['label']}: {field['reason']}")

## Test 5: Completion Percentage Tracker

In [ ]:
# Track progress as user fills out form
progressive_data = [
    {},  # Empty
    {"project_name": "Community Center"},  # 1 field
    {"project_name": "Community Center", "project_description": "Description here"},  # 2 fields
]

print("Form Completion Progress:")
for i, data in enumerate(progressive_data):
    percentage = completeness_validation_service.get_completion_percentage(form_schema, data)
    bar_length = int(percentage / 5)  # Scale to 20 chars
    bar = "█" * bar_length + "░" * (20 - bar_length)
    print(f"  Step {i+1}: [{bar}] {percentage}%")

## Test 6: Conditional File Upload Requirements

In [ ]:
# Form with conditional MOV requirement
conditional_form = {
    "fields": [
        {
            "field_id": "has_external_funding",
            "field_type": "radio_button",
            "label": "Does this project have external funding?",
            "required": True,
            "options": [
                {"label": "Yes", "value": "yes"},
                {"label": "No", "value": "no"}
            ]
        },
        {
            "field_id": "funding_documents",
            "field_type": "file_upload",
            "label": "Upload Funding Documents",
            "required": False,
            "conditional_mov_requirement": {
                "field_id": "has_external_funding",
                "operator": "equals",
                "value": "yes"
            }
        }
    ]
}

# Scenario 1: Said YES but no files uploaded
data_yes_no_files = {"has_external_funding": "yes"}
result1 = completeness_validation_service.validate_completeness(
    conditional_form, data_yes_no_files, uploaded_movs=[]
)

print("Scenario 1: Answered YES but no documents uploaded")
print(f"  Is Complete: {result1['is_complete']}")
if not result1['is_complete']:
    print(f"  Missing: {result1['missing_fields'][0]['label']}")

# Scenario 2: Said YES and files uploaded
data_yes_with_files = {"has_external_funding": "yes"}
mock_movs = [{"filename": "funding_agreement.pdf"}]
result2 = completeness_validation_service.validate_completeness(
    conditional_form, data_yes_with_files, uploaded_movs=mock_movs
)

print("\nScenario 2: Answered YES and documents uploaded")
print(f"  ✅ Is Complete: {result2['is_complete']}")

# Scenario 3: Said NO (files not required)
data_no = {"has_external_funding": "no"}
result3 = completeness_validation_service.validate_completeness(
    conditional_form, data_no, uploaded_movs=[]
)

print("\nScenario 3: Answered NO (files not required)")
print(f"  ✅ Is Complete: {result3['is_complete']}")

## Test 7: End-to-End Workflow Example

This shows how all three services work together in a typical workflow.

In [ ]:
print("🔄 End-to-End Workflow Simulation\n")

# Step 1: BLGU fills out form
print("Step 1: BLGU User fills out assessment form")
form_data = {
    "project_name": "Barangay Health Program",
    "project_description": "Community health initiative",
    "completion_rate": 82.0,
    "required_documents": ["doc1", "doc2", "doc3", "doc4"]
}
print(f"  ✏️ Form data entered")

# Step 2: Check completeness (before allowing submission)
print("\nStep 2: System checks if all required fields are filled")
completeness_result = completeness_validation_service.validate_completeness(
    form_schema, form_data
)
if completeness_result['is_complete']:
    print(f"  ✅ Form is complete! Ready to submit.")
else:
    print(f"  ❌ Form incomplete. Missing: {completeness_result['missing_fields']}")
    print("  Cannot submit yet!")

# Step 3: Calculate compliance (after submission, by system)
print("\nStep 3: System calculates compliance status")
compliance_status = calculation_engine_service.execute_calculation(
    complex_schema, form_data
)
print(f"  📊 Calculated Status: {compliance_status.value}")

# Step 4: Generate remark
print("\nStep 4: System generates feedback remark")
remark = calculation_engine_service.get_remark_for_status(
    remark_schema, compliance_status
)
print(f"  💬 Remark: {remark}")

# Step 5: Save to database (ComplianceValidationService would handle this)
print("\nStep 5: Results saved to database")
print(f"  💾 validation_status: {compliance_status.value}")
print(f"  💾 generated_remark: {remark}")

print("\n✅ Workflow Complete!")

## 🎯 Summary

You've now tested all three Epic 1.0 services:

✅ **CalculationEngineService**
- Evaluates calculation rules
- Supports 6 rule types
- Generates Pass/Fail/Conditional status

✅ **CompletenessValidationService**
- Checks if required fields are filled
- Handles conditional requirements
- Tracks completion percentage

✅ **ComplianceValidationService**
- Orchestrates calculation + database updates
- Bulk validation support
- Schema recalculation

### Next Steps:
1. Create API endpoints (Epic 2.0)
2. Build frontend forms (Epic 3.0)
3. Test end-to-end with real users
